If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets as well as other dependencies. Uncomment the following cell and run it.

In [106]:
# ! pip install datasets evaluate transformers rouge-score nltk

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

Then you need to install Git-LFS. Uncomment the following instructions:

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [107]:
import transformers

# print(transformers.__version__)

You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/seq2seq).

We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

# Fine-tuning a model on a summarization task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model for a summarization task. We will use the [XSum dataset](https://arxiv.org/pdf/1808.08745.pdf) (for extreme summarization) which contains BBC articles accompanied with single-sentence summaries.

![Widget inference on a summarization task](https://github.com/huggingface/notebooks/blob/main/examples/images/summarization.png?raw=1)

We will see how to easily load the dataset for this task using 🤗 Datasets and how to fine-tune a model on it using the `Trainer` API.

In [108]:
model_checkpoint = "t5-larg"

This notebook is built to run  with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a sequence-to-sequence version in the Transformers library. Here we picked the [`t5-small`](https://huggingface.co/t5-small) checkpoint.

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [109]:
from datasets import load_dataset
from evaluate import load

# raw_datasets = load_dataset("xsum")
raw_dataset_train = load_dataset("json",data_files="/home/ritvik/QuestionGeneration/T5/data/final/train.json")
raw_dataset_test = load_dataset("json",data_files="/home/ritvik/QuestionGeneration/T5/data/final/test.json")
raw_dataset_val = load_dataset("json",data_files="/home/ritvik/QuestionGeneration/T5/data/final/val.json")

metric_rouge = load("rouge")
metric_bleu = load("bleu")

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [110]:
raw_dataset = raw_dataset_train
raw_dataset['test'] = raw_dataset_test['train']
raw_dataset['validation'] = raw_dataset_val['train']

To access an actual element, you need to select a split first, then give an index:

In [111]:
# raw_datasets["train"][0]

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [112]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [113]:
show_random_elements(raw_dataset["train"])

,target_text,prefix,input_text
0,Was Subramanian Swamy present at the press briefing held by the opposition parties in December 2018?,generate question,Subramanian Swamy spotted at a press briefing of the Opposition parties and has officially joined them.
1,Henley passport index,generate question,"Nigeria now has the fifth most powerful passport, after CNN report on countries easing travel curbs."
2,BJP MLA harassed woman beaten up video,generate question,Video BJP MLA beaten up for harassing a woman
3,HHS whistleblower against hydroxychloroquine,generate question,"“So the so-called HHS Whistleblower was against HYDROXYCHLOROQUINE. Then why did he make, and sign, an emergency use authorization? ... A dumb @60Minutes hit job on a grandstanding Never Trumper!”"
4,Sinovac's COVID-19 vaccine efficacy,generate question,The Sinovac vaccine is only 50 percent effective against COVID-19.


The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

You can call its `compute` method with your predictions and labels, which need to be list of decoded strings:

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that the model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [114]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/ritvik/conda/envs/simpletransformers/lib/python3.9/site-packages/huggingface_hub/utils/_er │
│ rors.py:259 in hf_raise_for_status                                                               │
│                                                                                                  │
│   256 │   </Tip>                                                                                 │
│   257 │   """                                                                                    │
│   258 │   try:                                                                                   │
│ ❱ 259 │   │   response.raise_for_status()                                                        │
│   260 │   except HTTPError as e:                                                                 │
│   261 │   │   error_code = response.headers.get("X-Error-Code")                                  │
│   262                                                                                            │
│                                                                                                  │
│ /home/ritvik/conda/envs/simpletransformers/lib/python3.9/site-packages/requests/models.py:1021   │
│ in raise_for_status                                                                              │
│                                                                                                  │
│   1018 │   │   │   )                                                                             │
│   1019 │   │                                                                                     │
│   1020 │   │   if http_error_msg:                                                                │
│ ❱ 1021 │   │   │   raise HTTPError(http_error_msg, response=self)                                │
│   1022 │                                                                                         │
│   1023 │   def close(self):                                                                      │
│   1024 │   │   """Releases the connection back to the pool. Once this method has been            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
HTTPError: 404 Client Error: Not Found for url: https://huggingface.co/t5-larg/resolve/main/tokenizer_config.json

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/ritvik/conda/envs/simpletransformers/lib/python3.9/site-packages/transformers/utils/hub.py │
│ :417 in cached_file                                                                              │
│                                                                                                  │
│    414 │   user_agent = http_user_agent(user_agent)                                              │
│    415 │   try:                                                                                  │
│    416 │   │   # Load from URL or cache if already cached                                        │
│ ❱  417 │   │   resolved_file = hf_hub_download(                                                  │
│    418 │   │   │   path_or_repo_id,                                                              │
│    419 │   │   │   filename,                                                                     │
│    420 │   │   │   subfolder=None if len(subfolder) == 0 else subfolder,                         │
│                                                                                                  │
│ /home/ritvik/conda/envs/simpletransformers/lib/python3.9/site-packages/huggingface_hub/utils/_va │
│ lidators.py:120 in _inner_fn                                                                     │
│                                                                             

By default, the call above will use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library.

You can directly call this tokenizer on one sentence or a pair of sentences:

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [115]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [pre + inp for inp,pre in zip(examples["input_text"],examples["prefix"]) ]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["target_text"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [116]:
# preprocess_function(raw_dataset['train'][:2])

To apply this function on all the pairs of sentences in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [117]:
tokenized_datasets = raw_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/122173 [00:00<?, ? examples/s]

Map:   0%|          | 0/30543 [00:00<?, ? examples/s]

Map:   0%|          | 0/38180 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [118]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/ritvik/conda/envs/simpletransformers/lib/python3.9/site-packages/huggingface_hub/utils/_er │
│ rors.py:259 in hf_raise_for_status                                                               │
│                                                                                                  │
│   256 │   </Tip>                                                                                 │
│   257 │   """                                                                                    │
│   258 │   try:                                                                                   │
│ ❱ 259 │   │   response.raise_for_status()                                                        │
│   260 │   except HTTPError as e:                                                                 │
│   261 │   │   error_code = response.headers.get("X-Error-Code")                                  │
│   262                                                                                            │
│                                                                                                  │
│ /home/ritvik/conda/envs/simpletransformers/lib/python3.9/site-packages/requests/models.py:1021   │
│ in raise_for_status                                                                              │
│                                                                                                  │
│   1018 │   │   │   )                                                                             │
│   1019 │   │                                                                                     │
│   1020 │   │   if http_error_msg:                                                                │
│ ❱ 1021 │   │   │   raise HTTPError(http_error_msg, response=self)                                │
│   1022 │                                                                                         │
│   1023 │   def close(self):                                                                      │
│   1024 │   │   """Releases the connection back to the pool. Once this method has been            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
HTTPError: 404 Client Error: Not Found for url: https://huggingface.co/t5-larg/resolve/main/config.json

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/ritvik/conda/envs/simpletransformers/lib/python3.9/site-packages/transformers/utils/hub.py │
│ :417 in cached_file                                                                              │
│                                                                                                  │
│    414 │   user_agent = http_user_agent(user_agent)                                              │
│    415 │   try:                                                                                  │
│    416 │   │   # Load from URL or cache if already cached                                        │
│ ❱  417 │   │   resolved_file = hf_hub_download(                                                  │
│    418 │   │   │   path_or_repo_id,                                                              │
│    419 │   │   │   filename,                                                                     │
│    420 │   │   │   subfolder=None if len(subfolder) == 0 else subfolder,                         │
│                                                                                                  │
│ /home/ritvik/conda/envs/simpletransformers/lib/python3.9/site-packages/huggingface_hub/utils/_va │
│ lidators.py:120 in _inner_fn                                                                     │
│                                                                                       

In [119]:
batch_size = 32
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
)

In [120]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [121]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result_rouge = metric_rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    result_bleu = metric_bleu.compute(predictions=decoded_preds, references=decoded_labels)

    # Extract a few results (Why do we multiply by 100, keep it a decimal)
    # result_rouge = {key: value * 100 for key, value in result_rouge.items()}
    # result_bleu = {key: value * 100 for key, value in result_bleu.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]

    result_rouge["gen_len"] = np.mean(prediction_lens)
    result_bleu["gen_len"] = np.mean(prediction_lens)

    print("Inside compute_metrics")
    # print(result_bleu, "\n",result_rouge)
    bleu_rouge_score = {"rouge1":result_rouge['rouge1'],"rouge2":result_rouge['rouge2'],"rougeL":result_rouge['rougeL'],"rougeLsum":result_rouge['rougeLsum'], 'bleu':result_bleu['bleu']}
    return {k: round(v, 4) for k, v in bleu_rouge_score.items()}

In [122]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"].shuffle(seed=42).select(range(100)),
    eval_dataset=tokenized_datasets["validation"].shuffle(seed=42).select(range(20)),
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [123]:
trainer.train()

/home/ritvik/conda/envs/simpletransformers/lib/python3.9/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/ritvik/conda/envs/simpletransformers/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Bleu
1,No log,4.005591,0.213700,0.122100,0.206900,0.205800,0.061900


Inside compute_metrics


TrainOutput(global_step=1, training_loss=4.295493125915527, metrics={'train_runtime': 0.7344, 'train_samples_per_second': 136.165, 'train_steps_per_second': 1.362, 'total_flos': 1374565171200.0, 'train_loss': 4.295493125915527, 'epoch': 1.0})

In [124]:
input_ids = tokenizer("generate question: Joe Biden is the president of the United States.", return_tensors="pt").to("cuda")
print(input_ids['input_ids'])

tensor([[3806,  822,   10, 4967, 2106,  537,   19,    8, 2753,   13,    8,  907,
         1323,    5,    1]], device='cuda:0')


In [125]:
outputs = model.generate(input_ids['input_ids'])

In [126]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Joe Biden
